In [4]:
import cfbd
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
import sqlite3
import DataFunctions


configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = '3WCU5V2X05Rvh60ZxUG8FarJN4s2D1lcd2c2r6Kz/qL1Y3tVBJtWsuNATnzHRV2h'
configuration.api_key_prefix['Authorization'] = 'Bearer'

## Transforming data to 

In [5]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
year=2022
division="fbs"
try:
    api_response = api_instance.get_games(year=year,division=division)
    print(len(api_response))
except:
    print(ApiException)
    

<class 'cfbd.rest.ApiException'>


In [3]:
#removing non fbs vs fbs games
i=0
while i < len(api_response):
    if(api_response[i].home_division==api_response[i].away_division=="fbs"
       and api_response[i].completed==True
       and api_response[i].neutral_site==False):
        i+=1
        continue
    else:
        api_response.remove(api_response[i])

print(len(api_response))

NameError: name 'api_response' is not defined

In [ ]:
gamelist = DataFunctions.get_fbs_games(api_instance=api_instance,year=2022)
len(gamelist)

In [ ]:
game0 = api_response[0]
game0str = str(game0).split("\n")
game0str = [s[2:-1] for s in game0str]
game0str = [s.replace("'","") for s in game0str]
game0str = [s.split(": ") for s in game0str]
game0dict = {s[0]:s[1] for s in game0str}
game0keys = list(game0dict.keys())

In [ ]:
gamemetrics = ['away_conference','away_id','away_points','away_team',
 'home_conference','home_id','home_points','home_team', 'id', 'season','neutral_site']

In [ ]:
for k in (set(game0keys)-set(gamemetrics)):
    game0dict.pop(k,None)

game0dict

In [ ]:
df_games = pd.DataFrame([game0dict])
df_games = df.set_index('id')
df_games = df.reset_index()
df_games

SPACE

In [ ]:
api_instance1 = cfbd.BettingApi(cfbd.ApiClient(configuration))
api_response1 = api_instance1.get_lines(year=year)

In [ ]:
api_response1[0]

In [ ]:
conferences=[]
for game in api_response:
    conferences.append(game.away_conference)
conferences=set(conferences)

conferences

In [ ]:
i=0
while i < len(api_response1):
    if(api_response1[i].away_conference in conferences
      and api_response1[i].home_conference in conferences):
        i+=1
        continue
    else:
        api_response1.remove(api_response1[i])

api_response1[0]